In [7]:
import numpy as np
import time
import os
from tqdm import tqdm
import cv2
from google.colab.patches import cv2_imshow
import json
import requests

upload face json


In [ ]:
data = []
with open('/content/face_detection.json') as f:
    for line in f:
        data.append(json.loads(line))
data

CLONE THE GIT REPO

In [4]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 13728 (delta 2), reused 2 (delta 0), pack-reused 13714
Receiving objects: 100% (13728/13728), 12.37 MiB | 12.26 MiB/s, done.
Resolving deltas: 100% (9356/9356), done.


STEPS:


1.   **Create file yolo-obj.cfg with the same content as in yolov4-custom.cfg**
2.   **Create file obj.names in the directory build\darknet\x64\data\, with objects names - each in new line**
3.   **Create file obj.data in the directory build\darknet\x64\data\, containing (where classes = number of objects)**
4.   **Put image-files (.jpg) of your objects in the directory build\darknet\x64\data\obj\**



PUTTING ALL IMAGES AND LABELS IN /content/darknet/build/darknet/x64/data/obj FOLDER

In [8]:
d = '/content/darknet/build/darknet/x64/data/obj'
try:  
  os.mkdir(d)
except:pass
count=0

count =0
for ob in tqdm(data[:100]):
  img = cv2.imdecode(np.array(bytearray(requests.get(ob['content']).content),np.uint8),-1)
  if img.shape[2]==4:
    continue
  count+=1
  image_anotate=[]
  f =open(f"{d}/{count}.txt",'a') 
  for content in ob['annotation']:
    x,y = content['points'][0].values()
    w,h = content['points'][1].values()
    f.write(f'0 {x+(w-x)/2} {y+(h-y)/2} {w-x} {h-y}\n')
  f.close()
  cv2.imwrite(f'/{d}/{count}.jpg',img)

100%|██████████| 100/100 [00:51<00:00,  1.95it/s]


Create file train.txt in directory build\darknet\x64\data\, with filenames of your images, each filename in new line, with path relative to darknet.exe, for example containing:

In [9]:
with open('/content/darknet/build/darknet/x64/data/train.txt','w') as f:
  for i in os.listdir('/content/darknet/build/darknet/x64/data/obj'):
    if i.endswith('.jpg'):
      f.write('/content/darknet/build/darknet/x64/data/obj/'+i+'\n')

Download pre-trained weights for the convolutional layers and put to the directory build\darknet\x64

build DARKNET

In [ ]:
%cd /content/darknet/ 
!bash build.sh

UPLOAD THE YOLO TINE PRETRAINED WEIGHTS to /content/darknet/build/darknet/x64 - yolov3-tiny.conv.11

Start Training the weights will save at the backup folder

In [ ]:
!./darknet detector train /content/darknet/build/darknet/x64/data/obj.data /content/darknet/build/darknet/x64/cfg/yolov3-tiny_custom.cfg /content/darknet/build/darknet/x64/yolov3-tiny.conv.11 -dont_show